<a href="https://colab.research.google.com/github/AdithyaaVenkatesh/Oasis_intern/blob/main/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJECT 3 - LEVEL 1**

---

# **Cleaning Data**

## 1) Data collection and Integrity

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats  # Ensure this import is included

df = pd.read_csv("/content/AB_NYC_2019.csv")


## 2) Missing Data Handling

In [ ]:
print(df.isnull().sum())

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [ ]:
df['last_review'].fillna("2019-07-08", inplace=True)
df['host_name'].fillna("Unknown", inplace=True)
df.fillna({'name': 'Unknown', 'neighbourhood_group': 'Unknown'}, inplace=True)
df['reviews_per_month'].fillna(0, inplace=True)
print(df.isnull().sum())

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64


<ipython-input-39-8c638653cb20>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['last_review'].fillna("2019-07-08", inplace=True)
<ipython-input-39-8c638653cb20>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

## 3) Duplicate Removal

In [ ]:
print("duplicate rows:")
print(df.duplicated().sum())

duplicate rows:
0


## 4) Standardization

In [ ]:
data = df.columns.str.lower().str.strip().str.replace(" ", "_")
print(data)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


In [ ]:
df['host_name'] = df['host_name'].str.strip().str.lower()
df['name'] = df['name'].str.title()  # Capitalize first letter of each word
print(df[['host_name', 'name']])


           host_name                                               name
0               john                 Clean & Quiet Apt Home By The Park
1           jennifer                              Skylit Midtown Castle
2          elisabeth                The Village Of Harlem....New York !
3        lisaroxanne                    Cozy Entire Floor Of Brownstone
4              laura   Entire Apt: Spacious Studio/Loft By Central Park
...              ...                                                ...
48890        sabrina    Charming One Bedroom - Newly Renovated Rowhouse
48891        marisol      Affordable Room In Bushwick/East Williamsburg
48892  ilgar & aysel            Sunny Studio At Historical Neighborhood
48893            taz               43Rd St. Time Square-Cozy Single Bed
48894     christophe  Trendy Duplex In The Very Heart Of Hell'S Kitchen

[48895 rows x 2 columns]


## 5) Outlier Detection

In [ ]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # 25th percentile
    Q3 = df[column].quantile(0.75)  # 75th percentile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Detect outliers for key columns
outliers_price = detect_outliers_iqr(df, 'price')
outliers_min_nights = detect_outliers_iqr(df, 'minimum_nights')
outliers_reviews = detect_outliers_iqr(df, 'reviews_per_month')
outliers_availability = detect_outliers_iqr(df, 'availability_365')

print("Outliers in price column:\n", outliers_price)
print("Outliers in minimum_nights column:\n", outliers_min_nights)
print("Outliers in reviews_per_month column:\n", outliers_reviews)
print("Outliers in availability_365 column:\n", outliers_availability)




Outliers in price column:
              id                                               name    host_id  \
61        15396                 Sunny & Spacious Chelsea Apartment      60278   
85        19601                Perfect For A Family Or Small Group      74303   
103       23686  2000 Sf 3Br 2Bath West Village Private  Townhouse      93790   
114       26933  2 Br / 2 Bath Duplex Apt With Patio! East Village      72062   
121       27659                   3 Story Town House In Park Slope     119588   
...         ...                                                ...        ...   
48758  36420289    Rustic Garden House Apt, 2 Stops From Manhattan   73211393   
48833  36450896   Brand New 3-Bed Apt In The Best Location Of Fidi   29741813   
48839  36452721  Massage Spa. Stay Overnight. Authors Artist Dr...  274079964   
48842  36453160  Luxury Manhattan Penthouse+Hudson River+Empire...  224171371   
48856  36457700  Large 3 Bed, 2 Bath , Garden , Bbq , All You Need   66993395   



In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers for the columns
df_cleaned = remove_outliers(df, 'price')
df_cleaned = remove_outliers(df_cleaned, 'minimum_nights')
df_cleaned = remove_outliers(df_cleaned, 'reviews_per_month')
df_cleaned = remove_outliers(df_cleaned, 'availability_365')



In [ ]:

df_cleaned['z_score_price'] = np.abs(stats.zscore(df_cleaned['price']))
df_cleaned['z_score_min_nights'] = np.abs(stats.zscore(df_cleaned['minimum_nights']))
df_cleaned['z_score_reviews'] = np.abs(stats.zscore(df_cleaned['reviews_per_month']))
df_cleaned['z_score_availability'] = np.abs(stats.zscore(df_cleaned['availability_365']))

outliers_z = df_cleaned[df_cleaned[['z_score_price', 'z_score_min_nights', 'z_score_reviews', 'z_score_availability']].gt(3).any(axis=1)]
print("\nOutliers detected using Z-score:\n", outliers_z)




Outliers detected using Z-score:
              id                                               name    host_id  \
4          5022   Entire Apt: Spacious Studio/Loft By Central Park       7192   
27         9357                               Midtown Pied-A-Terre      30193   
71        17037                            Lovely Ev Artist'S Home      66035   
116       26969                2 Story Family Home In Williamsburg     115307   
225       58059                Private Room On Historic Sugar Hill     277379   
...         ...                                                ...        ...   
48664  36373544   Large Studio With Tall Ceiling Near Central Park   35141789   
48665  36374141    Sunny Bedroom With Renovated Rooftop + Balcony!   37872024   
48700  36389944                Family Friendly Apt In Midtown East  224309949   
48794  36428255  Skyscraper Ultimate Luxury At The Heart Of Bklyn.  148289089   
48892  36485431            Sunny Studio At Historical Neighborhood   23492

## 6) save cleaned data

In [ ]:
df_cleaned.to_csv("/content/AB_NYC_2019_cleaned.csv", index=False)

print("\nOutlier handling completed successfully! The cleaned data is saved.")


Outlier handling completed successfully! The cleaned data is saved.
